In [1]:
!pip install lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 KB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=084a3ddbe736bf6d84d7922da8aa0d9dd3ff2c7ff25dfd0cec894742cde43a5c
  Stored in directory: /root/.cache/pip/wheels/ed/d7/c9/5a0130d06d6310bc6cbe55220e6e72dcb8c4eff9a478717066
Successfully built lime


In [2]:
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, mean_squared_error, recall_score, roc_curve
from sklearn.utils import shuffle, check_random_state
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from google.colab import drive, files
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd 
import numpy as np 
import pickle
import lime
import lime.lime_tabular
import matplotlib.pyplot as plt

In [3]:
drive.mount('/content/drive')

ben  = pd.read_csv('/content/drive/My Drive/ISM/LEMNA-master/benign.csv')
mal  = pd.read_csv('/content/drive/My Drive/ISM/LEMNA-master/malicious.csv')
data = ben.append(mal, ignore_index=True)
data = shuffle(data)

Mounted at /content/drive


In [75]:
y = data['class']
x = data.drop(['class'], axis=1)
x = x.drop(['filename'],axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.3, random_state=27)

In [73]:
x

,author_dot,author_lc,author_len,author_mismatch,author_num,author_oth,author_uc,box_nonother_types,box_other_only,company_mismatch,...,subject_oth,subject_uc,title_dot,title_lc,title_len,title_mismatch,title_num,title_oth,title_uc,version
3503,1,10,14,1,0,2,2,240,0,0,...,28,19,1,25,46,67,8,9,4,6
7301,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,3
8675,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3
1741,0,0,0,0,0,0,0,10,0,0,...,0,0,0,0,0,0,0,0,0,7
6669,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5347,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,5
3236,0,5,6,2,0,0,1,64,0,1,...,4,5,0,21,32,26,0,5,6,4
5294,0,6,7,1,0,1,0,0,1,0,...,1,0,0,6,7,1,0,1,0,6
1083,0,8,8,1,0,0,0,6,0,0,...,0,0,1,37,56,1,8,9,2,4


In [77]:
feature_means = []
feature_maxs = []

def categorical_helper_fun(x):
    if isinstance(x, int) or isinstance(x,float):
        return int(x)
    else:
        return 0

print (feature_maxs)

[]


In [78]:
for i in data:
    if True in data[i]:
        data[i] = list(map(lambda x: categorical_helper_fun(x), data[i]))
    feature_means.append(np.mean(data[i]))
    feature_maxs.append(max(data[i]))

feature_names = []
for i in data:
    feature_names.append(i)
print (feature_names)

['class', 'filename', 'author_dot', 'author_lc', 'author_len', 'author_mismatch', 'author_num', 'author_oth', 'author_uc', 'box_nonother_types', 'box_other_only', 'company_mismatch', 'count_acroform', 'count_acroform_obs', 'count_action', 'count_action_obs', 'count_box_a4', 'count_box_legal', 'count_box_letter', 'count_box_other', 'count_box_overlap', 'count_endobj', 'count_endstream', 'count_eof', 'count_font', 'count_font_obs', 'count_image_large', 'count_image_med', 'count_image_small', 'count_image_total', 'count_image_xlarge', 'count_image_xsmall', 'count_javascript', 'count_javascript_obs', 'count_js', 'count_js_obs', 'count_obj', 'count_objstm', 'count_objstm_obs', 'count_page', 'count_page_obs', 'count_startxref', 'count_stream', 'count_stream_diff', 'count_trailer', 'count_xref', 'createdate_dot', 'createdate_mismatch', 'createdate_ts', 'createdate_tz', 'createdate_version_ratio', 'creator_dot', 'creator_lc', 'creator_len', 'creator_mismatch', 'creator_num', 'creator_oth', 'cr

In [79]:
feature_means

[0.49994999499949994,
 0.0,
 0.028102810281028102,
 2.5554555455545556,
 5.558155815581558,
 0.45484548454845486,
 0.3311331133113311,
 0.9034903490349034,
 1.768076807680768,
 7.6995699569957,
 0.3698369836983698,
 0.039003900390039,
 0.26852685268526855,
 0.0007000700070007,
 0.8547854785478548,
 0.0018001800180018,
 0.012001200120012,
 0.39503950395039505,
 7.291529152915292,
 32.18431843184318,
 0.001000100010001,
 95.8006800680068,
 30.436043604360435,
 1.551055105510551,
 14.646364636463646,
 0.0035003500350035003,
 0.11071107110711072,
 0.46524652465246524,
 0.9158915891589159,
 3.6563656365636565,
 0.00030003000300030005,
 2.1642164216421644,
 0.877987798779878,
 0.0039003900390039005,
 0.714071407140714,
 0.024402440244024402,
 100.96999699969997,
 1.572057205720572,
 0.0,
 5.9126912691269125,
 0.008000800080008,
 1.5372537253725373,
 30.58715871587159,
 0.15111511151115112,
 1.4382438243824383,
 1.3857385738573857,
 0.0,
 0.48044804480448045,
 374569570.7540754,
 -3008.974397

In [80]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 3503 to 335
Columns: 137 entries, class to version
dtypes: int64(137)
memory usage: 10.8 MB


In [81]:
x_train=np.asarray(x_train).astype(np.float32)
x_test=np.asarray(x_test).astype(np.float32)

In [82]:
# model = Sequential()
# model.add(Dense(1024, input_dim=len(x.columns), activation='relu'))
# model.add(Dense(1024, activation='relu'))
# model.add(Dense(1024, activation='sigmoid'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(x_train,y_train, epochs=20, batch_size=10)
# y_pred = model.predict(x_test)
# y_pred = list(map(lambda x: x>=0.5, y_pred))
# accuracy_score(y_test, y_pred)
# pickle.dump(model, open('/content/drive/My Drive/ISM/LEMNA-master/classifier.sav', 'wb'))

In [83]:
model = pickle.load(open('/content/drive/My Drive/ISM/LEMNA-master/classifier.sav', 'rb'))

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-03-03 20:33:34         2266
metadata.json                                  2023-03-03 20:33:34           64
variables.h5                                   2023-03-03 20:33:34     26904280
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...

In [84]:
l = lime.lime_tabular.LimeTabularExplainer(np.array(x_train), feature_names = feature_names, class_names=['True', 'False'], discretize_continuous=True)

In [85]:
def data_inverse(l, data_row, num_samples):
    """Generates a neighborhood around a prediction.
    For numerical features, perturb them by sampling from a Normal(0,1) and
    doing the inverse operation of mean-centering and scaling, according to
    the means and stds in the training data. For categorical features,
    perturb by sampling according to the training distribution, and making
    a binary feature that is 1 when the value is the same as the instance
    being explained.
    Args:
        data_row: 1d numpy array, corresponding to a row
        num_samples: size of the neighborhood to learn the linear model
    Returns:
        A tuple (data, inverse), where:
            data: dense num_samples * K matrix, where categorical features
            are encoded with either 0 (not equal to the corresponding value
            in data_row) or 1. The first row is the original instance.
            inverse: same as data, except the categorical features are not
            binary, but categorical (as the original data)
    """
    data = np.zeros((num_samples, data_row.shape[0]))
    categorical_features = range(data_row.shape[0])
    if l.discretizer is None:
        data = l.random_state.normal(
            0, 1, num_samples * data_row.shape[0]).reshape(
            num_samples, data_row.shape[0])
        if l.sample_around_instance:
            data = data * l.scaler.scale_ + data_row
        else:
            data = data * l.scaler.scale_ + l.scaler.mean_
        categorical_features = l.categorical_features
        first_row = data_row
    else:
        first_row = l.discretizer.discretize(data_row)
    data[0] = data_row.copy()
    inverse = data.copy()
    for column in categorical_features:
        values = l.feature_values[column]
        freqs = l.feature_frequencies[column]
        inverse_column = l.random_state.choice(values, size=num_samples,
                                                  replace=True, p=freqs)
        binary_column = np.array([1 if x == first_row[column]
                                  else 0 for x in inverse_column])
        binary_column[0] = 1
        inverse_column[0] = data[0, column]
        data[:, column] = binary_column
        inverse[:, column] = inverse_column
    if l.discretizer is not None:
        inverse[1:] = l.discretizer.undiscretize(inverse[1:])
    inverse[0] = data_row
    return data, inverse

array([[ 0.,  0.,  0., ...,  0.,  0.,  3.],
       [ 0.,  0., 11., ...,  7.,  2.,  7.],
       [ 1.,  8., 14., ...,  2., 12.,  4.],
       ...,
       [ 0.,  4.,  4., ..., 13., 10.,  4.],
       [ 0.,  0.,  0., ...,  0.,  0.,  3.],
       [ 0.,  0.,  0., ...,  0.,  0.,  3.]], dtype=float32)

In [96]:
a = data_inverse(l, x_train[23], 1000)
gm = GaussianMixture(n_components=2, covariance_type = 'diag')

p_data_train = a[1]

p_data_test = list(map(lambda x: x>=0.5, model.predict(p_data_train)))

gm.fit(p_data_train, p_data_test)

components = gm.precisions_

ind = []
target = gm.predict(x_train[0].reshape(1,-1))[0]
for i in range(len(p_data_train)):
    if gm.predict(p_data_train[i].reshape(1,-1))[0]==target:
        ind.append(i)

feature_count = np.zeros(135)

for i in ind:
    feature_count = list(map(int, a[0][i]!=0))+feature_count

feature_count.shape

32/32 [==============================] - 0s 6ms/step


(135,)

In [97]:
feature_count

array([85., 23., 15., 46., 81., 68., 66., 43., 36., 98., 82., 99., 42.,
       98., 97., 97., 38., 24., 98., 27., 26., 28., 22., 99., 96., 13.,
       89., 21., 99., 99., 17., 97., 39., 96., 29., 88., 99., 23., 99.,
       28., 26., 96., 31., 25., 99., 26., 25., 15., 26., 22., 37., 31.,
       44., 25., 42., 22., 19., 82., 85., 22., 98., 90., 90., 89., 83.,
       90., 93., 23., 30., 18., 26., 28., 28., 99., 29., 26.,  9., 15.,
       99., 91., 40., 37., 28., 97., 21., 99., 86., 38., 35., 33., 97.,
       20., 39., 82., 86., 82., 99., 99., 99., 99., 99., 99., 26., 24.,
       27., 99., 99., 99., 23., 30., 34., 14., 17., 25., 25., 12., 22.,
       18., 32., 33., 99., 79., 75., 79., 89., 68., 86., 15., 20., 29.,
       33., 73., 19., 26., 19.])

(array([ 11,  23,  28,  29,  36,  38,  44,  73,  78,  85,  96,  97,  98,
         99, 100, 101, 105, 106, 107, 120]),)

In [ ]:
feature_count

In [ ]:
def fidelity_test1(model, x_train, feature_count):
    '''
    If features Fx are accurately selected, then removing Fx from
    the input x will lead to classifying this image to a different
    label, i.e., “shoe” 
        '''
    o = []
    n = []
    new_prob = []
    for x_train_point in x_train:
        original = model.predict(x_train_point.reshape(1,-1))>=0.5
        len_ind = 0
        ma = max(feature_count)
        for i in range(len(feature_count)):
            if feature_count[i] == ma and x_train_point[i]!=0:
                x_train_point[i] = 0
                len_ind += 1
            if len_ind == 30:
                break
        np = model.predict(x_train_point.reshape(1,-1))
        new_prob.append(np)
        o.append(original)
        n.append(np>=0.5)
    return o, n, new_prob

def fidelity_test2(model, x_train, feature_count):
    '''
    If features Fx are accurately selected, then adding the feature values of Fx
    to an image of “shoe” is likely to lead to a
    misclassification, i.e., classifying it as a “sweater”
    '''
    new_prob = []
    orig_prob = []
    len_ind = 0
    o = []
    n = []
    ma = max(feature_count)
    ind = []
    for i in range(len(feature_count)):
        #feature_count[i] == ma and 
        if x_train[0][i]==0 and feature_count[i] == ma-200:
            ind.append(i)
            len_ind += 1
        if len_ind == 30:
            break
    for x_train_point in x_train:
        original = model.predict(x_train_point.reshape(1,-1))
        op = original
        original = original<=0.5
        if original==True:
            original = 1
        else:
            original = 0
        if sum(x_train_point[ind]==0)==len_ind and original!=target:
            for i in range(len_ind):
                x_train_point[ind[i]] = feature_maxs[ind[i]]
            new_predict = model.predict(x_train_point.reshape(1,-1))
            new_prob.append(new_predict)
            new_predict = new_predict>=0.5
            orig_prob.append(op)
            o.append(original)
            n.append(new_predict)

    return new_prob, new_prob, o, n

def fidelity_test3(model, feature_count):
    '''
    If features Fx are accurately selected, we can craft a synthetic
    images that only contains the features in Fx, and this synthetic 
    image is likely to be classified as “sweater”
    '''
    len_ind = 0
    o = []
    n = []
    ma = max(feature_count)
    ind = []
    for i in range(len(feature_count)):
        if feature_count[i] == ma:
            ind.append(i)
            len_ind += 1
        if len_ind == 30:
            break
    x_train_points = [0]*len(feature_count)
    for i in ind:
        x_train_points[i] = feature_means[i]
    return model.predict(np.array(x_train_points).reshape(1,-1))

In [ ]:
o1, n1, n_prob = fidelity_test1(model, x_train[0].reshape(1,-1), feature_count)
orig_prob, ft2_prob, o2, n2 = fidelity_test2(model, x_train[0].reshape(1,-1), feature_count)

print (model.predict(x_train[0].reshape(1,-1)))
print (fidelity_test3(model, feature_count))

y_pred=model.predict(p_data_train)

gm_pred1=[]
gm_pred2=[]


In [ ]:
def LocalApproxAccuracy():      
    for i in range(len(y_pred)):
        gm_pred1.append(gm.predict_proba(p_data_train)[i][1])
        gm_pred2.append(gm.predict_proba(p_data_train)[i][0])
    rmse1=mean_squared_error(y_pred,gm_pred1)
    rmse2=mean_squared_error(y_pred,gm_pred2)
    return(min(rmse1,rmse2))

In [ ]:
#LIME Prediction 
pred1=[]
for i in range(len(y_pred)):
    pred1.append(l.explain_instance(p_data_train[i],gm.predict_proba, num_features=5).predict_proba[0])
    print (i)
rmse=mean_squared_error(y_pred,pred1)

LocalApproxAccuracy()

def precision(ytrue,ypred):
    return precision_score(ytrue,ypred)
def recall(ytrue,ypred):
    return recall_score(ytrue,ypred)
def roc(ytrue,yscore):         
    fpr, tpr, thresholds = roc_curve(ytrue,yscore)
    plt.figure()
#     print (fpr[2], tpr[2], thresholds, sep='\n')
    plt.plot(fpr, tpr, color='darkorange')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic ')
    plt.show()
def confusionMatrix(ytrue,ypred):
    return confusion_matrix(ytrue,ypred)

for i in range(len(n2)):
    n2[i] = n2[i][0][0]

for i in range(len(o1)):
    o1[i]=o1[i][0][0]
    n1[i]=n1[i][0][0]

    print(precision(o1,n1))
    print(recall(o1,n1))
    print (confusionMatrix(o1, n1))

for i in range(len(o1)):
    if o1[i]==True:
        o1[i]=1
    else:
        o1[i]=0
for i in range(len(n_prob)):
    n_prob[i] = n_prob[i][0][0]

roc(o1, n_prob)

for i in range(len(o2)):
    n2[i] = n2[i][0][0]

print(precision(o2,n2))
print(recall(o2,n2))
print (confusionMatrix(o2, n2))

for i in range(len(ft2_prob)):
    ft2_prob[i] = ft2_prob[i][0][0]

print(roc(o2,ft2_prob))